In [2]:
import pandas as pd
import numpy as np
import matplotlib.pyplot as plt
from pydataset import data

from env import host, username, password

url = f'mysql+pymysql://{username}:{password}@{host}/employees'

In [3]:
pd.read_sql('SELECT * FROM employees LIMIT 5 OFFSET 50', url)

,emp_no,birth_date,first_name,last_name,gender,hire_date
0,10051,1953-07-28,Hidefumi,Caine,M,1992-10-15
1,10052,1961-02-26,Heping,Nitsch,M,1988-05-21
2,10053,1954-09-13,Sanjiv,Zschoche,F,1986-02-04
3,10054,1957-04-04,Mayumi,Schueller,M,1995-03-13
4,10055,1956-06-06,Georgy,Dredge,M,1992-04-27


# Exercise 1
Load the mpg dataset. Read the documentation for it, and use the data to answer these questions:
**On average, which manufacturer has the best miles per gallon?
How many different manufacturers are there?
How many different models are there?
Do automatic or manual cars have better miles per gallon?

In [4]:
mpg = data('mpg')
data('mpg', show_doc=True)

mpg

PyDataset Documentation (adopted from R Documentation. The displayed examples are in R)

## Fuel economy data from 1999 and 2008 for 38 popular models of car

### Description

This dataset contains a subset of the fuel economy data that the EPA makes
available on http://fueleconomy.gov. It contains only models which had a new
release every year between 1999 and 2008 - this was used as a proxy for the
popularity of the car.

### Usage

    data(mpg)

### Format

A data frame with 234 rows and 11 variables

### Details

  * manufacturer. 

  * model. 

  * displ. engine displacement, in litres 

  * year. 

  * cyl. number of cylinders 

  * trans. type of transmission 

  * drv. f = front-wheel drive, r = rear wheel drive, 4 = 4wd 

  * cty. city miles per gallon 

  * hwy. highway miles per gallon 

  * fl. 

  * class. 




In [5]:
mpg.head()

,manufacturer,model,displ,year,cyl,trans,drv,cty,hwy,fl,class
1,audi,a4,1.8,1999,4,auto(l5),f,18,29,p,compact
2,audi,a4,1.8,1999,4,manual(m5),f,21,29,p,compact
3,audi,a4,2.0,2008,4,manual(m6),f,20,31,p,compact
4,audi,a4,2.0,2008,4,auto(av),f,21,30,p,compact
5,audi,a4,2.8,1999,6,auto(l5),f,16,26,p,compact


In [22]:
# On average, which manufacturer has the best miles per gallon?

mpg['avg_mpg'] = (mpg.cty + mpg.hwy) / 2
best_mpg = mpg.iloc[mpg.avg_mpg.idxmax()]
print(best_mpg.manufacturer)

volkswagen


In [23]:
# How many different manufacturers are there?

manu_count = mpg.manufacturer.value_counts().count()
print(manu_count)

15


In [24]:
# How many different models are there?

model_count = mpg.model.value_counts().count()
print(model_count)

38


In [27]:
# Do automatic or manual cars have better miles per gallon?

automatics = mpg[mpg.trans.str.contains('auto')]

manuals = mpg[mpg.trans.str.contains('manual')]

#print(automatics.avg_mpg.mean())
#print(manuals.avg_mpg.mean())

print('Manual cars have better average gas mileage.')

Manual cars have better average gas mileage.


# Exercise 2 - Joining & Merging

**Copy the users and roles dataframes from the examples above. What do you think a right join would look like? An outer join? What happens if you drop the foreign keys from the dataframes and try to merge them?

In [28]:
users = pd.DataFrame({
    'id': [1, 2, 3, 4, 5, 6],
    'name': ['bob', 'joe', 'sally', 'adam', 'jane', 'mike'],
    'role_id': [1, 2, 3, 3, np.nan, np.nan]
})
users

,id,name,role_id
0,1,bob,1.0
1,2,joe,2.0
2,3,sally,3.0
3,4,adam,3.0
4,5,jane,NaN
5,6,mike,NaN


In [29]:
roles = pd.DataFrame({
    'id': [1, 2, 3, 4],
    'name': ['admin', 'author', 'reviewer', 'commenter']
})
roles

,id,name
0,1,admin
1,2,author
2,3,reviewer
3,4,commenter


In [32]:
# Right join

pd.merge(users, roles, left_on='role_id', right_on='id', how='right')

,id_x,name_x,role_id,id_y,name_y
0,1.0,bob,1.0,1,admin
1,2.0,joe,2.0,2,author
2,3.0,sally,3.0,3,reviewer
3,4.0,adam,3.0,3,reviewer
4,NaN,NaN,NaN,4,commenter


In [33]:
# Outer join

pd.merge(users, roles, left_on='role_id', right_on='id', how='outer')

,id_x,name_x,role_id,id_y,name_y
0,1.0,bob,1.0,1.0,admin
1,2.0,joe,2.0,2.0,author
2,3.0,sally,3.0,3.0,reviewer
3,4.0,adam,3.0,3.0,reviewer
4,5.0,jane,NaN,NaN,NaN
5,6.0,mike,NaN,NaN,NaN
6,NaN,NaN,NaN,4.0,commenter


In [39]:
# What happens if you drop the foreign keys from the dataframes and try to merge them?

users_test = users.drop(columns='role_id')
roles_test = roles.drop(columns='id')

pd.merge(users_test, roles_test)

,id,name


In [40]:
print('It shows only the columns id and name, and does not include any rows.')

It shows only the columns id and name, and does not include any rows.


# Exercise 3 - Getting data from SQL databases

* a. Create a function named get_db_url. It should accept a username, hostname, password, and database name and return a url formatted like in the examples in this lesson.
* b. Use your function to obtain a connection to the employees database.
* c. Once you have successfully run a query:
  * Intentionally make a typo in the database url. What kind of error message do you see?
  * Intentionally make an error in your SQL query. What does the error message look like?
* d. Read the employees and titles tables into two separate dataframes
* e. Visualize the number of employees with each title.
* f. Join the employees and titles dataframes together.
* g. Visualize how frequently employees change titles.
* h. For each title, find the hire date of the employee that was hired most recently with that title.
* i. Write the code necessary to create a cross tabulation of the number of titles by department. (Hint: this will involve a combination of SQL and python/pandas code)